In [13]:
from face_detector import YoloFaceDetector
import numpy as np
import cv2

In [14]:
model = YoloFaceDetector(target_size=720,gpu=-1,min_face=90)

-1
c:\Bigdata\yolo_dnn_exercise\weights/yolov5n_state_dict.pt


In [17]:
cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    
    bboxes,points = model.predict(img)
    print(bboxes)
    
    for el in bboxes:
        for bbox in el:
            cv2.rectangle(img, (bbox[0],bbox[1]),(bbox[2],bbox[3]),(255,0,0),1)
        
    cv2.imshow("Webcam", img) # This will open an independent window
    
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
        
cv2.destroyAllWindows()

[[[227, 85, 401, 317]]]
[[[228, 84, 402, 318]]]
[[[228, 78, 405, 322]]]
[[[228, 85, 404, 319]]]
[[[228, 83, 403, 324]]]
[[[227, 83, 402, 318]]]
[[[229, 82, 404, 318]]]
[[[230, 84, 405, 322]]]
[[[235, 82, 412, 323]]]
[[[237, 86, 413, 322]]]
[[[237, 84, 412, 323]]]
[[[236, 80, 412, 321]]]
[[[236, 85, 412, 321]]]
[[[235, 80, 413, 322]]]
[[[237, 83, 414, 322]]]
[[[258, 78, 433, 311]]]
[[[283, 82, 458, 317]]]
[[[14, 250, 91, 346], [321, 79, 503, 327]]]
[[[336, 90, 514, 327]]]
[[[348, 86, 522, 319]]]
[[[351, 88, 522, 320]]]
[[[349, 92, 521, 324]]]
[[[349, 87, 522, 323]]]
[[[348, 87, 522, 320]]]
[[[349, 91, 522, 324]]]
[[[351, 91, 523, 324]]]
[[[352, 93, 523, 324]]]
[[[351, 90, 523, 324]]]
[[[350, 91, 522, 324]]]
[[[350, 90, 523, 325]]]
[[[349, 87, 523, 323]]]
[[[349, 91, 522, 324]]]
[[[349, 89, 522, 321]]]
[[[349, 89, 523, 322]]]
[[[348, 91, 521, 324]]]
[[[348, 92, 520, 322]]]
[[[347, 91, 520, 324]]]
[[[346, 90, 520, 326]]]
[[[347, 91, 519, 325]]]
[[[346, 90, 520, 326]]]
[[[346, 89, 520, 325